In [2]:
# pip install jupyter_contrib_nbextensions
# pip install pyspark
# pip install pandas
# pip install confluent_kafka

In [1]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer
import pandas as pd
import os
import json

In [5]:
KAFKA_HOST = "kafka:9092"

data = {
    # "products" : pd.read_csv("./data/products.csv"),
    "orders"   : pd.read_csv("./data/orders.csv")
}

admin = AdminClient({ "bootstrap.servers" : KAFKA_HOST })

In [6]:
for my_topic, df in data.items():

    
    #### delete topic
    print(f"Deleting topic {my_topic} if exists")
    ops = admin.delete_topics([my_topic], operation_timeout=30)
    for topic, f in ops.items():
        try:
            f.result()
            print(f"Topic {topic} is deleted")
        except Exception as e:
            print(f"Failed to delete topic {topic}: {e}")

    #### recreate topic
    print(f"Creating topic {my_topic}")
    new_topics = [NewTopic(my_topic, num_partitions=3, replication_factor=1)]
    ops = admin.create_topics(new_topics)

Deleting topic orders if exists
Failed to delete topic orders: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Broker: Unknown topic or partition"}
Creating topic orders


In [7]:
for my_topic, df in data.items():

    # import data to Kafka
    producer = Producer({ "bootstrap.servers" : KAFKA_HOST })
    print(f"Importing {my_topic} data")

    cnt = 0
    batch_size = 5e4
    for _, row in df.iterrows():
        if cnt % batch_size==0 and cnt!=0:
            producer.flush()
            print(f"{cnt} message is imported")
        producer.produce(my_topic, json.dumps(row.to_dict()))
        cnt+=1
    producer.flush()

Importing orders data
50000 message is imported
100000 message is imported
150000 message is imported
200000 message is imported
250000 message is imported
300000 message is imported
350000 message is imported
400000 message is imported
450000 message is imported
500000 message is imported
550000 message is imported
600000 message is imported
650000 message is imported
700000 message is imported
750000 message is imported
800000 message is imported
850000 message is imported
900000 message is imported
950000 message is imported
